# Which borough has the most noise complaints (or, more selecting data)

## Install

In [ ]:
SYFT_VERSION = ">=0.9,<1.0.0"
package_string = f'"syft{SYFT_VERSION}"'
# %pip install {package_string} -q

In [ ]:
# syft absolute
import syft as sy

sy.requires(SYFT_VERSION)

In [ ]:
server = sy.orchestra.launch(name="pandas-test-datasite-3", port=7083, reset=True)

# Data owner: upload data

In [ ]:
datasite_client = server.login(email="info@openmined.org", password="changethis")

In [ ]:
# The usual preamble
%matplotlib inline
# third party
import matplotlib.pyplot as plt
import pandas as pd

# syft absolute
from syft.service.project.project import Project
from syft.util.util import PANDAS_DATA

# Make the graphs a bit prettier, and bigger
plt.style.use("ggplot")

# This is necessary to show lots of columns in pandas 0.12.
# Not necessary in pandas 0.13.
pd.set_option("display.width", 5000)
pd.set_option("display.max_columns", 60)

plt.rcParams["figure.figsize"] = (15, 5)

## Load data

We're going to use a new dataset here, to demonstrate how to deal with larger datasets. This is a subset of the of 311 service requests from NYC Open Data.

In [ ]:
# because of mixed types we specify dtype to prevent any errors
complaints = pd.read_csv(
    sy.autocache(f"{PANDAS_DATA}/311-service-requests.csv"), dtype="unicode"
)

Depending on your pandas version, you might see an error like "DtypeWarning: Columns (8) have mixed types". This means that it's encountered a problem reading in our data. In this case it almost certainly means that it has columns where some of the entries are strings and some are integers.

For now we're going to ignore it and hope we don't run into a problem, but in the long run we'd need to investigate this warning.

In [ ]:
complaints

In [ ]:
assert len(complaints) == 111069

## Create Mock data

In [ ]:
# stdlib
import random
from random import randint

# syft absolute
from syft.serde.mock import CachedFaker

Let's create the mock data for the complaint dataset.

In [ ]:
fake = CachedFaker()

In [ ]:
fake_functions = {
    "Unique Key": lambda x: randint(1, 1000000),
    "Location": lambda x: (fake.coordinate(), fake.coordinate()),
    "Agency": lambda x: random.choice(["NYPD", "DOHMH", "DPR"]),
    "X Coordinate (State Plane)": lambda x: randint(1, 1000000),
    "Y Coordinate (State Plane)": lambda x: randint(1, 1000000),
    "Complaint Type": lambda x: random.choice(
        ["Illegal Parking", "Noise - Street/Sidewalk", "'Animal in a Park'"]
    ),
    "Descriptor": lambda x: random.choice(
        [
            "Branch or Limb Has Fallen Down",
            "Branches Damaged",
            "Broken Fence",
            "Broken Glass",
        ]
    ),
    "School Number": lambda x: random.choice(
        [
            "B073",
            "B077",
            "B079",
            "B080-01",
            "B087",
            "B099",
            "B100",
            "B102",
            "B109",
            "B111",
        ]
    ),
    "Bridge Highway Segment": lambda x: random.choice(
        [
            "Grand Central Pkwy (Exit 1 E-W)",
            "Grand Central Pkwy (Exit 10) - 69th Rd-Jewel Ave (Exit 11)",
            "GrandCentral Pkwy/VanWyck Expwy/College Point Blvd (Exit 22 A-E)",
            "Hamilton Ave (Exit 2A) - Gowanus Expwy (I-278) (Exit 1)",
            "Harding Ave (Exit 9) - Throgs Neck Br",
        ]
    ),
}

In [ ]:
fake_triggers = {
    "Street": lambda x: fake.street_name(),
    "Date": lambda x: fake.date_time(),
    "Long": lambda x: fake.coordinate(),
    "Lat": lambda x: fake.coordinate(),
    "Address": lambda x: fake.address(),
    "Name": lambda x: fake.name(),
    "City": lambda x: fake.city(),
    "Zip": lambda x: fake.zipcode(),
}

In [ ]:
mock_data = {}
for col in complaints.columns:
    col_vals = complaints[col]

    if col in fake_functions:
        mock_func = fake_functions[col]
    elif len(set(complaints[col])) < 100:
        values = list(set(complaints[col]))
        mock_func = lambda x: random.choice(values)  # noqa: E731,B023
    else:
        for trigger in fake_triggers.keys():
            if trigger in col:
                mock_func = fake_triggers[trigger]
    mock_data[col] = [mock_func(None) for x in range(len(complaints))]

In [ ]:
mock = pd.DataFrame(data=mock_data)

In [ ]:
mock.head()

In [ ]:
dataset = sy.Dataset(
    name="bikes",
    asset_list=[
        sy.Asset(name="complaints", data=complaints, mock=mock, mock_is_real=False)
    ],
)
datasite_client.upload_dataset(dataset)

## Create data scientist

In [ ]:
user = datasite_client.register(
    name="Jane Doe",
    email="jane@caltech.edu",
    password="abc123",
    password_verify="abc123",
    institution="Caltech",
    website="https://www.caltech.edu/",
)

# todo: give user data scientist role

guest_datasite_client = server.client

guest_client = guest_datasite_client.login(email="jane@caltech.edu", password="abc123")

# Data scientist: Create syft_function

## Download mock and submit project

### Get mock

In [ ]:
guest_datasite_client = server.client
guest_client = guest_datasite_client.login(email="jane@caltech.edu", password="abc123")

In [ ]:
ds = guest_datasite_client.datasets[0]

In [ ]:
asset = ds.assets["complaints"]

In [ ]:
complaints = asset.mock

### Selecting only noise complaints


I'd like to know which borough has the most noise complaints. First, we'll take a look at the data to see what it looks like:

In [ ]:
complaints[:5]

To get the noise complaints, we need to find the rows where the "Complaint Type" column is "Noise - Street/Sidewalk". I'll show you how to do that, and then explain what's going on.



In [ ]:
noise_complaints = complaints[complaints["Complaint Type"] == "Noise - Street/Sidewalk"]
noise_complaints[:3]

If you look at noise_complaints, you'll see that this worked, and it only contains complaints with the right complaint type. But how does this work? Let's deconstruct it into two pieces

In [ ]:
complaints["Complaint Type"] == "Noise - Street/Sidewalk"

This is a big array of Trues and Falses, one for each row in our dataframe. When we index our dataframe with this array, we get just the rows where our boolean array evaluated to True. It's important to note that for row filtering by a boolean array the length of our dataframe's index must be the same length as the boolean array used for filtering.

You can also combine more than one condition with the & operator like this:

In [ ]:
is_noise = complaints["Complaint Type"] == "Noise - Street/Sidewalk"
in_brooklyn = complaints["Borough"] == "BROOKLYN"
complaints[is_noise & in_brooklyn][:5]

Or if we just wanted a few columns:



In [ ]:
complaints[is_noise & in_brooklyn][
    ["Complaint Type", "Borough", "Created Date", "Descriptor"]
][:10]

### A digression about numpy arrays

On the inside, the type of a column is pd.Series

In [ ]:
# third party
import numpy as np
import pandas as pd

In [ ]:
pd.Series([1, 2, 3])

and pandas Series are internally numpy arrays. If you add .values to the end of any Series, you'll get its internal numpy array

In [ ]:
np.array([1, 2, 3])

In [ ]:
pd.Series([1, 2, 3]).values

So this binary-array-selection business is actually something that works with any numpy array:

In [ ]:
arr = np.array([1, 2, 3])

In [ ]:
arr != 2

In [ ]:
arr[arr != 2]

### So, which borough has the most noise complaints?

In [ ]:
is_noise = complaints["Complaint Type"] == "Noise - Street/Sidewalk"
noise_complaints = complaints[is_noise]
noise_complaints["Borough"].value_counts()

It's the BRONX (for this Mock)! But what if we wanted to divide by the total number of complaints, to make it make a bit more sense? That would be easy too:

In [ ]:
noise_complaint_counts = noise_complaints["Borough"].value_counts()
complaint_counts = complaints["Borough"].value_counts()

In [ ]:
noise_complaint_counts

In [ ]:
noise_complaint_counts / complaint_counts

Oops, why was that zero? That's no good. This is because of integer division in Python 2. Let's fix it, by converting complaint_counts into an array of floats.

In [ ]:
noise_complaint_counts / complaint_counts.astype(float)

In [ ]:
(noise_complaint_counts / complaint_counts.astype(float)).plot(kind="bar")

So Bronx really does complain more about noise than the other boroughs in our mock! Neat.

## Request real result

Now that we finished our analysis on the mock data, we can request this execution on the real data.

In [ ]:
@sy.syft_function(
    input_policy=sy.ExactMatch(complaints=asset.pointer),
    output_policy=sy.SingleExecutionExactOutput(),
)
def get_counts(complaints):
    is_noise = complaints["Complaint Type"] == "Noise - Street/Sidewalk"
    noise_complaints = complaints[is_noise]
    noise_complaint_counts = noise_complaints["Borough"].value_counts()
    complaint_counts = complaints["Borough"].value_counts()
    return noise_complaint_counts / complaint_counts.astype(float)

Create and submit project

In [ ]:
new_project = sy.Project(
    name="Pandas chapter 3",
    description="Hi, I would like to plot the histogram of the noise complaint counts per area.",
    members=[guest_client],
)
new_project

In [ ]:
project = new_project.send()
assert isinstance(project, sy.service.project.project.Project)
project

In [ ]:
project.create_code_request(get_counts, guest_client)

In [ ]:
assert len(guest_client.code.get_all()) == 1

In [ ]:
assert len(project.events) == 1

In [ ]:
assert isinstance(project.events[0], sy.service.project.project.ProjectRequest)

# Data owner: approve request

## Get notifications

In [ ]:
datasite_client = server.login(email="info@openmined.org", password="changethis")

In [ ]:
notifications = datasite_client.notifications.get_all_unread()

In [ ]:
notifications

In [ ]:
project_notification = [
    x for x in notifications if issubclass(x.linked_obj.object_type, Project)
][0]

In [ ]:
request = project_notification.link.events[0].request
func = request.code

In [ ]:
func

In [ ]:
get_counts_user_func = func.run

In [ ]:
real_data = datasite_client.datasets[0].assets[0].data

In [ ]:
real_result = get_counts_user_func(complaints=real_data)

In [ ]:
real_result[:3]

In [ ]:
result = request.approve()
assert isinstance(result, sy.SyftSuccess)

# Data scientist: compute result

In [ ]:
asset = guest_client.datasets[0].assets[0]

In [ ]:
guest_client.code[0].status

In [ ]:
result_ptr = guest_client.code.get_counts(complaints=asset)
real_result = result_ptr.get()
real_result.plot(kind="bar")

In [ ]:
server.land()